In [1]:
import pandas as pd
from pathlib import Path
from joblib import Parallel, delayed
import itertools
import numpy as np
from tqdm import tqdm
from typing import List

In [2]:
workspace = Path("/mnt/wks3/aschneuwl/workspace")

In [4]:
milk_data_fpath = workspace / Path("data/preprocessed/dairy/k33.parquet")
farm_meta_fpath = workspace / Path("data/preprocessed/dairy/swiss_farms_anonymzed.parquet")
calvings_fpath = workspace / Path("data/preprocessed/dairy/k11.parquet")

In [7]:
samples_df = pd.read_parquet(milk_data_fpath)
farm_meta_df = pd.read_parquet(farm_meta_fpath)
calvings_df = pd.read_parquet(calvings_fpath)

In [8]:
samples_df.sampleMethod.value_counts()

sampleMethod
1    155063501
2      3724956
3        72883
Name: count, dtype: int64

In [11]:
sorted(list(samples_df.columns))

['aceton',
 'acetonIr',
 'acetonMmol',
 'alpAltitude',
 'animalBreedCode',
 'animalId',
 'animalName',
 'bhbConcentration',
 'calvingDate',
 'caseinMeasured',
 'citrate',
 'comment',
 'dataOrigin',
 'farmId',
 'farmIdLocationSample',
 'farmIdTvd',
 'farmIdTvdSample',
 'fat',
 'fat2',
 'fatMeasured',
 'fatMeasured2',
 'herdIdentification',
 'labCode',
 'lactationNumber',
 'lactose',
 'milk',
 'milkEvening',
 'milkMorning',
 'milkUreaNitrogen',
 'milkingMethod',
 'milkingTimeEvening',
 'milkingTimeMorning',
 'protein',
 'protein2',
 'proteinMeasured',
 'proteinMeasured2',
 'recordType',
 'recordVersion',
 'registrationLbe',
 'registrationMbk',
 'sampleMethod',
 'sampleNumber',
 'samplePersistence',
 'sampleWeighingDate',
 'somaticCellCount',
 'source',
 'weighingCode',
 'weighingType',
 'year']

In [12]:
years = list(samples_df.year.unique())
breeds =  list(samples_df.animalBreedCode.unique())

In [13]:
years

[1973,
 1974,
 1975,
 1976,
 1978,
 1979,
 1980,
 1981,
 1982,
 1983,
 1984,
 1985,
 1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022,
 2023,
 2024]

In [14]:
breeds

['HO', 'XX', 'SI', 'BS', 'OB', '60', '70', 'SF', 'RH', 'HR', 'RF', 'JE']

In [ ]:
samples_df[samples_df.animalBreedCode == "XX"]

In [16]:
def iqa(df: pd.DataFrame, column: str):
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    return lower_bound, upper_bound

In [17]:
def iqa_filter(year: int, breed: str, columns: List[str], fpath: Path, min_samples_per_year: int = 1000):
    df = pd.read_parquet(fpath, filters=[[("year", "==", year), ("animalBreedCode", "==", breed)]], engine="pyarrow")
    df = df.dropna(subset=columns, how='any')
    
    filters = []
    for column in columns:
        lower_bound, upper_bound = iqa(df, column)
    
        if (not np.isnan(lower_bound)) and (not np.isnan(upper_bound)):
            filter_str = f"((year == {year}) & (animalBreedCode == '{breed}') & ({column} > {lower_bound}) & ({column} < {upper_bound}))"
            filters.append(filter_str)

    if len(filters) != 0:
        df = df.query(" & ".join(filters))
    else:
        df = None
    
    if (df is not None) and (df.shape[0] < min_samples_per_year):
        df = None

    return df

In [28]:
def filter_yield_for_variable(years: List[int],
                              breeds: List[str],
                              columns: List[str],
                              samples_fpath: Path,
                              farm_meta_df: pd.DataFrame,
                              calvings_df: pd.DataFrame,
                              secondary: List[str]) -> pd.DataFrame:
    """
    Function to filter out outliers from the data (parallelized) . above and below whisker
    """

    # Apply the IQA filter for each breed and year
    combs = list(itertools.product(years, breeds))
    filtered = Parallel(n_jobs=-1)(delayed(iqa_filter)(y,b,columns, samples_fpath) for (y,b) in tqdm(combs))
    filtered = [s for s in filtered if s is not None]

    # base columns
    df_cols = ["animalBreedCode", "animalId", "calvingDate", "farmIdLocationSample", "lactationNumber", "year", "milkingMethod", "sampleWeighingDate", "sampleMethod", "weighingType"]
    
    # primary performance variables
    df_cols.extend(columns)
    
    # secondary performance variables (no IQA applied or na check)
    df_cols.extend(secondary)
    
    df_filtered = pd.concat(filtered)[df_cols]

    # create DIM variable
    df_filtered["days_in_milk"] = df_filtered["sampleWeighingDate"] - df_filtered["calvingDate"]

    #df_filtered = df_filtered.loc[df_filtered["days_in_milk"].dt.days >= 0, :]

    df_filtered = pd.merge(df_filtered, farm_meta_df[["zip", "altitude", "locationType", "farmId"]], left_on='farmIdLocationSample', right_on="farmId", how='left')

    # drop all the samples which were not assigned to a farm
    df_filtered = df_filtered[df_filtered["zip"].notna()].drop(["farmId"], axis=1)

    df_filtered = df_filtered.dropna(subset=["lactationNumber", "year", "milkingMethod", "sampleWeighingDate", "days_in_milk"])

    #df_filtered = pd.merge(df_filtered, calvings_df[["calfId", "calvingDate"]], left_on='animalId', right_on="calfId", how='left')
    #df_filtered["dateOfBirth"] = df_filtered["calvingDate_y"]
    #df_filtered.drop("calvingDate_y", axis=1, inplace=True)
    #df_filtered["calvingDate"] = df_filtered["calvingDate_x"]
    #df_filtered.drop("calvingDate_x", axis=1, inplace=True)
    #df_filtered["age"] = df_filtered["sampleWeighingDate"] - df_filtered["dateOfBirth"]

    #df_filtered["farmId"] = df_filtered["farmId_y"]

    #df_filtered.drop("farmId_y", axis=1, inplace=True)

    #df_filtered.drop("calfId", axis=1, inplace=True)

    df_filtered.to_parquet(workspace / Path(f"/data/preprocessed/dairy/k33_{"_".join(columns)}_iqa_filtered.parquet"), partition_cols=["year"])

    return df_filtered

# Milk

In [16]:
samples_df.milk.isna().sum()

0

In [17]:
samples_df.fat.isna().sum()

389753

In [19]:
samples_df.protein.isna().sum()

360494

In [21]:
samples_df.somaticCellCount.isna().sum()

23119246

In [22]:
samples_df.milkUreaNitrogen.isna().sum()

23899843

In [23]:
samples_df.bhbConcentration.isna().sum()

105720727

In [24]:
samples_df.aceton.isna().sum()

106428744

In [27]:
samples_df.aceton.isna().sum()

106428744

In [26]:
samples_df.acetonIr.isna().sum()

107230933

In [25]:
samples_df.lactose.isna().sum()

412700

In [ ]:
filter_yield_for_variable(years, breeds, primary_dairy_performance_vars, fpath, farm_meta_df, calvings_df)

# Primary Performance Variables

In [21]:
primary_dairy_performance_vars = ["milk", "fat", "protein"]
secondary = ["milkUreaNitrogen", "bhbConcentration", "aceton", "acetonMmol", "acetonIr", "somaticCellCount", "lactose"]

In [ ]:
perf_vars =  filter_yield_for_variable(years, breeds, primary_dairy_performance_vars, milk_data_fpath, farm_meta_df, calvings_df, secondary)

In [39]:
perf_vars.shape

(132963491, 24)